> In this notebook, we import and format the UIS education statistics for modeling. Features from this data serve to measure "school climate".

In [232]:
import pandas as pd
import numpy as np

In [233]:
uis_1 = pd.read_csv('../data/UIS/EDULIT_DS_23082019165936814.csv')
uis_2 = pd.read_csv('../data/UIS/EDULIT_DS_25082019015237837.csv')

uis = pd.concat([uis_1, uis_2], sort=True)
uis.head()

,Country,EDULIT_IND,Flag Codes,Flags,Indicator,LOCATION,TIME,Time,Value
0,Cyprus,AIR_1,+,National Estimation,Gross intake ratio to Grade 1 of primary educa...,CYP,2000,2000,100.69412
1,Cyprus,AIR_1,+,National Estimation,Gross intake ratio to Grade 1 of primary educa...,CYP,2001,2001,99.80127
2,Cyprus,AIR_1,+,National Estimation,Gross intake ratio to Grade 1 of primary educa...,CYP,2002,2002,98.23879
3,Cyprus,AIR_1,+,National Estimation,Gross intake ratio to Grade 1 of primary educa...,CYP,2003,2003,99.06412
4,Cyprus,AIR_1,+,National Estimation,Gross intake ratio to Grade 1 of primary educa...,CYP,2004,2004,100.39945


In [234]:
countries = ['Albania', 'Algeria', 'Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil',
             'Bulgaria', 'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia',
             'Cyprus', 'Czechia', 'Denmark', 'Dominican Republic', 'Estonia', 'Finland',
             'France', 'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Indonesia',
             'Ireland', 'Israel', 'Italy', 'Japan', 'Jordan', 'Kazakhstan', 'Latvia',
             'Lebanon', 'Lithuania', 'Luxembourg', 'Malaysia', 'Malta', 'Mexico',
             'Montenegro', 'Netherlands', 'New Zealand', 'North Macedonia', 'Norway',
             'Peru', 'Poland', 'Portugal', 'Qatar', 'Republic of Korea', 'Republic of Moldova',
             'Romania', 'Russian Federation', 'Singapore', 'Slovakia', 'Slovenia', 'Spain',
             'Sweden', 'Switzerland', 'Thailand', 'Trinidad and Tobago', 'Tunisia', 'Turkey',
             'United Arab Emirates', 'United Kingdom of Great Britain and Northern Ireland',
             'United States of America', 'Uruguay', 'Viet Nam']

In [235]:
features = [feature for feature in uis['Indicator'].unique()]
len(features)

97

In [236]:
uis_df = pd.DataFrame(index=None, columns=features)
for year in [2015]:
    year_filter = (uis['TIME'] == year)
    
    for country in countries:
        country_filter = (uis['Country'] == country)

        year_country_dict = {feature: 0 for feature in features}
        year_country_index = [f"{year}_{country}"]

        for feature in features:
            feature_filter = (uis['Indicator'] == feature)
            
            try:
                year_country_dict[feature] += uis[year_filter
                                                  & country_filter
                                                  & feature_filter]['Value'].values[0]
            except:
                year_country_dict[feature] = np.nan
                
        year_country_df = pd.DataFrame(year_country_dict, index=year_country_index)
        uis_df = pd.concat([uis_df, year_country_df])

In [237]:
good_feature_dict = {column: uis_df[column].isna().sum()
                     for column in uis_df.columns
                     if uis_df[column].isna().sum() < 30}
good_features = list(good_feature_dict.keys())
len(good_features)

40

In [238]:
years = [year for year in range(2000, 2016, 3)]
years

[2000, 2003, 2006, 2009, 2012, 2015]

In [239]:
uis_df = pd.DataFrame(index=None, columns=good_features)
for year in years:
    year_filter = (uis['TIME'] == year)
    
    for country in countries:
        country_filter = (uis['Country'] == country)

        year_country_dict = {feature: 0 for feature in good_features}
        year_country_index = [f"{year}_{country}"]

        for feature in good_features:
            feature_filter = (uis['Indicator'] == feature)
            
            try:
                year_country_dict[feature] += uis[year_filter
                                                  & country_filter
                                                  & feature_filter]['Value'].values[0]
            except:
                year_country_dict[feature] = np.nan
                
        year_country_df = pd.DataFrame(year_country_dict, index=year_country_index)
        uis_df = pd.concat([uis_df, year_country_df])

In [240]:
uis_df.shape

(408, 40)

In [242]:
uis_df.to_csv('../data/UIS/uis.csv', index_label='Year_Country')